### 1. Preliminray

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os
from itertools import product
from solver import train_resnet_with_cub
from solver import test_resnet_with_cub

KAGGLE_INPUT_DIR = "cub2002011/CUB_200_2011"
DATA_PATH = os.path.join("/kaggle", "input", KAGGLE_INPUT_DIR)

# NOTE: you need to uncomment the following code if the model is trained in local
DATA_PATH = os.path.join("data", "CUB_200_2011")

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

### 2. Grid-Search Optimal Learning Rate

#### 2.1 Set the range of hyper-parameters 

In [17]:
# set hyper-parameters here
num_epochs = [15]
fine_tuning_lrs = [5e-5, 1e-4, 5e-4]
output_lrs = [1e-3, 2e-3, 5e-3, 8e-3, 1e-2]

configurations = list(product(fine_tuning_lrs, output_lrs))

#### 2.2 Train

In [ ]:
best_accs = []

# train with the pretrained model
for config in configurations:
    curr_best_acc = train_resnet_with_cub(DATA_PATH, num_epochs, fine_tuning_lr=config[0], output_lr=config[1])
    best_accs.extend(curr_best_acc)

best_acc = max(best_accs)    
best_fine_tune_lr, best_output_lr = None, None

# write the results into a txt file
with open('best_accuracy_lr.txt', 'w') as f:
    f.write("Configuration        Accuracy\n")
    f.write("=" * 30 + "\n")
    for config, accuracy in zip(configurations, best_accs):
        # find the best config
        if accuracy == best_acc and best_fine_tune_lr is None:
            best_fine_tune_lr, best_output_lr = config
        # write the training record into file 
        f.write("({:>7.5f}, {:>7.5f})   {:>8.6f}\n".format(config[0], config[1], accuracy))

#### 2.3 Inspect the Accuracy

In [ ]:
# !more best_accuracy_lr.txt

### 3. Find the Optimal Training Epoch

In [ ]:
num_epochs = [15, 30, 45]

# init the configurations
best_accs = []

# train with the pretrained model
curr_best_acc = train_resnet_with_cub(DATA_PATH, num_epochs, best_fine_tune_lr, best_output_lr, save=True)
best_accs.extend(curr_best_acc)
    
# write the results into a txt file
with open('best_accuracy_ep.txt', 'w') as f:
    f.write("Epoch  Accuracy\n")
    f.write("=" * 50 + "\n")
    for num_epoch, accuracy in zip(num_epochs, best_accs):
        f.write("{:>2}     {:>8.6f}\n".format(num_epoch, accuracy))

### 4. Train with Random Initialized Weights

In [ ]:
curr_best_acc = train_resnet_with_cub(DATA_PATH, num_epochs, best_fine_tune_lr, best_output_lr, pretrain=False)

### 5. Test the trained model

In [16]:
MODEL_PATH = os.path.join("Fine_Grain", "resnet18_cub.pth")
test_resnet_with_cub(DATA_PATH, MODEL_PATH)

100%|██████████| 94/94 [02:50<00:00,  1.81s/it]


For the best model on the CUB dataset, Top-5 train accuracy is 1.000000, Top-1 train accuracy is 1.000000


100%|██████████| 91/91 [02:43<00:00,  1.80s/it]

For the best model on the CUB dataset, Top-5  test accuracy is 0.880911, Top-1  test accuracy is 0.639455
